In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.python.keras.datasets import mnist
from tensorflow.contrib.eager.python import tfe

D:\Users\Yue\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# enable eager mode
tf.enable_eager_execution()
tf.set_random_seed(0)
np.random.seed(0)

In [3]:
if not os.path.exists('weights/'):
    os.makedirs('weights/')

# constants
units = 128
batch_size = 100
epochs = 2
num_classes = 10

In [4]:
# dataset loading
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((-1, 28, 28))  # 28 timesteps, 28 inputs / timestep
x_test = x_test.reshape((-1, 28, 28))  # 28 timesteps, 28 inputs / timeste

# one hot encode the labels. convert back to numpy as we cannot use a combination of numpy
# and tensors as input to keras
y_train_ohe = tf.one_hot(y_train, depth=num_classes).numpy()
y_test_ohe = tf.one_hot(y_test, depth=num_classes).numpy()

print('x train', x_train.shape)
print('y train', y_train_ohe.shape)
print('x test', x_test.shape)
print('y test', y_test_ohe.shape)

x train (60000, 28, 28)
y train (60000, 10)
x test (10000, 28, 28)
y test (10000, 10)


# Canonical RNN 

This is the suggested way to use Keras RNNs - create the Cells, wrap with an RNN and then simply call the inputs on the RNN. 

However, in practice, this is several times slower than expected. The simplest explanation is the call to `K.rnn()` inside the RNN class, which is used to symbolically loop over timesteps when Tensorflow is not running in Eager mode.

This symbolic operation severely hampers the performance of the model. One approach to somewhat reduce this issue is to write the loop ourselves. There is an example in `06_03_rnn.ipynb`, where I take a LSTMCell and write the pythonic loop and call the cell on the slices of data manually. It is faster than the 1 layer version that is shown here, but still alot slower than the `BasicLSTM` that I wrote - which is just the LSTM layer in Eager code from scratch. 

The performance of BasicLSTM is almost the same, has none of the frills of the Keras LSTM but its 1 layer and 2 layer versions are significantly faster than the version presented here.

In [5]:
class RNN(tf.keras.Model):
    def __init__(self, units, num_classes, num_layers=1):
        super(RNN, self).__init__()
        self.cells = [tf.keras.layers.LSTMCell(units, implementation=2) for _ in range(num_layers)]  # Use GPU implementation
        self.rnn = tf.keras.layers.RNN(self.cells, unroll=True)  # extremely slow if not unrolled - probably because it is using K.rnn() internally.
        self.classifier = tf.keras.layers.Dense(num_classes)

    def call(self, inputs, training=None, mask=None):
        # this approach is canonical, but slow
        # probably will be fixed in later versions of Tensorflow
        
        x = self.rnn(inputs)
        output = self.classifier(x)

        # softmax op does not exist on the gpu, so always use cpu
        with tf.device('/cpu:0'):
            output = tf.nn.softmax(output)

        return output

In [6]:
device = '/cpu:0' if tfe.num_gpus() == 0 else '/gpu:0'

with tf.device(device):
    # build model and optimizer
    model = RNN(units, num_classes, num_layers=2)
    model.compile(optimizer=tf.train.AdamOptimizer(0.01), loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # TF Keras tries to use entire dataset to determine shape without this step when using .fit()
    # Fix = Use exactly one sample from the provided input dataset to determine input/output shape/s for the model
    dummy_x = tf.zeros((1, 28, 28))
    model.call(dummy_x)

    # train
    model.fit(x_train, y_train_ohe, batch_size=batch_size, epochs=epochs,
              validation_data=(x_test, y_test_ohe), verbose=1)

    # evaluate on test set
    scores = model.evaluate(x_test, y_test_ohe, batch_size, verbose=1)
    print("Final test loss and accuracy :", scores)

    saver = tfe.Saver(model.variables)
    saver.save('weights/06_01_rnn/weights.ckpt')

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 349s 6ms/step - loss: 0.3126 - acc: 0.9010 - val_loss: 0.1028 - val_acc: 0.9713
Epoch 2/2
10000/10000 [==============================] - 16s 2ms/step
Final test loss and accuracy : [0.08836719910963438, 0.9737000066041946]
